### YOLO V8 Plane Detector
* using directly YOLO v8 model
* process a video

In [1]:
import math
from os import path
from tqdm import tqdm
import cv2

from ultralytics import YOLO

In [2]:
# load model

# passati a modello large
MODEL_PATH = "./runs/detect/train3/weights/best.pt"

model = YOLO(MODEL_PATH)

# set model parameters
# model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides["iou"] = 0.45  # NMS IoU threshold
# model.overrides['agnostic_nms'] = False  # NMS class-agnostic
# model.overrides['max_det'] = 1000

# Video in input
VIDEO_PATH = "mixkit-jet1.mp4"

In [3]:
# VIDEO_PATH = "mixkit-plane-taking-off.mp4"
CODEC = "mp4v"

ONLY_NAME = VIDEO_PATH.split(".")[0]

# name of the video with BB
VIDEO_OUT = f"{ONLY_NAME}_bb.mp4"

# settings for the BB that will be added
# remember: OpenCV is BGR
color = (0, 0, 255)  # red
tickness = 1

In [4]:
# for the original video
cap = cv2.VideoCapture(VIDEO_PATH)

n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = math.ceil(cap.get(cv2.CAP_PROP_FPS))

print()
print("Analyzed input video:")
print()
print(f"Number of frames is: {n_frames}")
print(f"Fps are: {fps}")
print()


Analyzed input video:

Number of frames is: 549
Fps are: 30



#### Apply YOLO to single frames

In [5]:
print(f"Processing input video {VIDEO_PATH}...")
print()

# read the first frame and initialize
ret, frame = cap.read()

# take height, width from the first frame
height, width, layers = frame.shape

# the annotated (output) video
video = cv2.VideoWriter(VIDEO_OUT, cv2.VideoWriter_fourcc(*CODEC), fps, (width, height))

# process frame by frame
# tqdm for progress bar
with tqdm(total=n_frames) as pbar:
    while ret:
        # apply yolo to the single frame
        results = model.predict(frame, verbose=False)

        # make a copy to add bb
        new_image = frame.copy()

        for result in results:
            # move to cpu
            result = result.cpu()
            # cast to int
            boxes = result.boxes.xyxy.numpy().astype(int)

            for box in boxes:
                # add rectangle
                new_image = cv2.rectangle(
                    new_image, (box[0], box[1]), (box[2], box[3]), color, tickness
                )

        # write the annotated image in the video
        video.write(new_image)

        # update the progress bar
        pbar.update(1)

        # next frame
        ret, frame = cap.read()

# close the output
video.release()

# close the input
cap.release()

Processing input video mixkit-jet1.mp4...



100%|██████████| 549/549 [00:14<00:00, 39.13it/s]


In [ ]:
print()
print("Process video with YOLO v8 model correctly terminated.")
print()